In [111]:
!pip install langfuse --quiet
!pip install langchain langchain-experimental langchain-openai yfinance
!pip install python-dotenv
!pip install --upgrade langfuse

In [75]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env


True

In [68]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(api_version="2024-12-01-preview",model= os.getenv("AZURE_DEPLOYED_MODEL_NAME"))

In [112]:
from langfuse import Langfuse
lf = Langfuse(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"), 
    secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
    host=os.getenv("LANGFUSE_HOST")
)

In [97]:
prompt = """
    You are a financial news analyst, given the news summary about company {company_name}:
    "{news}"
    Classify the sentiment and extract entities into the following JSON format:
    {format_instructions}
    """
lf.create_prompt(
    name="market_sentiment_analyzer_prompt",
    prompt=prompt,
    labels=["production"]
)


In [117]:
import requests

def get_company_stock_code(company_name):
    url = "https://query1.finance.yahoo.com/v1/finance/search"
    params = {"q": company_name}
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.get(url, params=params, headers=headers)
    data = response.json()

    if data.get("quotes"):
        return data["quotes"][0]["symbol"]
    return "UNKNOWN"

get_company_stock_code("Apple")

'AAPL'

In [118]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool

def fetch_company_finance_news(stock_code):
    tool = YahooFinanceNewsTool()
    result = tool.invoke(stock_code)
    return result

fetch_company_finance_news('AAPL')

"Ex-Apple Engineers Behind $200M Xnor Deal Launch ElastixAI, Secure $16M To Revolutionize AI Inference Across Devices\nSeattle-based ElastixAI, founded just months ago by veteran engineers behind Apple's (NASDAQ:AAPL) $200 million acquisition of Xnor, has raised $16 million from top-tier investors, including Bellevue-based capital venture company FUSE. The stealth-mode startup with elite Apple pedigree is quietly tackling one of the most expensive pain points in artificial intelligence deployment: inference, GeekWire reports. Don't Miss: Hasbro, MGM, and Skechers trust this AI marketing firm — Invest before it's\n\nMark Zuckerberg Once Vented Why Apple AirPods Have No Competition: 'One Of The Protocols That They Used...'\nMeta Platforms Inc. (NASDAQ:META) CEO Mark Zuckerberg had once criticized the proprietary AirPods protocol of Apple Inc. (NASDAQ:AAPL), advocating for it to be made accessible to competitors. What Happened: Zuckerberg, on the Joe Rogan Podcast episode 2255, expressed 

In [79]:
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import PydanticOutputParser

class OutputSchema(BaseModel):
    company_name: str
    stock_code: str
    newsdesc: str
    sentiment: str
    people_names: List[str]
    places_names: List[str]
    other_companies_referred: List[str]
    related_industries: List[str]
    market_implications: str
    confidence_score: float

parser = PydanticOutputParser(pydantic_object=OutputSchema)

In [ ]:
from langchain_core.prompts import PromptTemplate

def sentiment_analyzer(company_name):
    trace = lf.trace(name="market_sentiment_analyzer", input={"company": company_name})

    span1 = trace.span(name="get_company_stock_code", input={"company_name": company_name})
    stock_code = get_company_stock_code(company_name)
    span1.end(output={"stock_code": stock_code})
    if stock_code == "UNKNOWN":
        return "UNKNOWN"

    span2 = trace.span(name="fetch_company_finance_news", input={"stock_code": stock_code})
    news = fetch_company_finance_news(stock_code)
    span2.end(output={"news": news})

    span3 = trace.span(name="analyze_sentiment", input={"news": news})

    prompt = PromptTemplate.from_template(
        lf.get_prompt("market_sentiment_analyzer_prompt").prompt
    )
    chain = prompt | llm | parser
    output = chain.invoke({
        "company_name": company_name,
        "news": news,
        "format_instructions": parser.get_format_instructions()
    })

    span3.end(output=output)

    return output.model_dump()

In [124]:
sentiment_analyzer("apple")

{'company_name': 'Apple Inc.',
 'stock_code': 'AAPL',
 'newsdesc': 'Ex-Apple Engineers Behind $200M Xnor Deal Launch ElastixAI, Secure $16M To Revolutionize AI Inference Across Devices. Meta Platforms Inc. (NASDAQ:META) CEO Mark Zuckerberg criticized the proprietary AirPods protocol of Apple Inc. (NASDAQ:AAPL). Dan Ives rebuked Trump’s announcement of a special tariff on Apple Inc. iPhones. Trump threatened Apple Inc. (NASDAQ:AAPL) with a 25% tariff for iPhones made outside the U.S.',
 'sentiment': 'mixed',
 'people_names': ['Mark Zuckerberg', 'Dan Ives', 'Donald Trump', 'Tim Cook'],
 'places_names': ['Seattle', 'India'],
 'other_companies_referred': ['Meta Platforms Inc.', 'FUSE', 'Xnor'],
 'related_industries': ['Artificial Intelligence',
  'Technology',
  'Manufacturing'],
 'market_implications': "Potential for increased production costs or tariffs affecting Apple's profitability. Opportunities in AI inference market from new startup.",
 'confidence_score': 0.75}

In [127]:
sentiment_analyzer("Google")

{'company_name': 'Alphabet Inc.',
 'stock_code': 'GOOG',
 'newsdesc': 'Investment bank TD Cowen, focusing on the potential of Alphabet Inc. (NASDAQ:GOOG)’s Waymo, kept a $195 price target and a Buy rating on GOOG recently. TD Cowen’s Analysis of Waymo The investment bank predicts that the U.S. gross bookings of Waymo, Alphabet Inc. (NASDAQ:GOOG)’s self-driving subsidiary, will jump to $6.1 billion in 2034. Disney files suit against YouTube for poaching executive, Bloomberg reports. Disney has filed a lawsuit against Alphabet’s (GOOG) YouTube, claiming breach of contract after YouTube hired longtime Disney executive Justin Connolly as its global head of media and sports, Bloomberg’s Lucas Shaw reports. Disney claims Connolly’s departure breaches his three-year contract and is seeking injunctions to stop his appointment, which will oversee YouTube’s relationships with major media companies and its live-sports portfolio.',
 'sentiment': 'mixed',
 'people_names': ['Justin Connolly', 'Lucas

In [129]:
sentiment_analyzer("Googgle")

HTTPError: HTTP Error 404: 